In [ ]:
import os 
import sys
import subprocess

REPO_LOCATION = 'https://github.com/stura36/deepsearch.git'
REPO_NAME = 'deepsearch'
REPO_BRANCH = 'main'
PACKAGES = []

# Clone the repository
#print('cloning the repository')
subprocess.call(['git', 'clone', '-b', REPO_BRANCH, REPO_LOCATION])
print("Start pulling")
#subprocess.call(['git','pull'])
# Install packages 
#print('installing packages')
#subprocess.call(['pip', 'install'] + PACKAGES)

# Setting env variables
sys.path.append(os.path.join("kaggle","working",REPO_NAME))

In [ ]:
!pip install lightning sentence-transformers

In [ ]:
import torch
from deepsearch.preprocess import read_data,find_max_token_len,create_text_model_input
from deepsearch.model_definition import tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
df = read_data()
#tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
raw_inputs = df["caption"].values.tolist()
token_max_len = find_max_token_len(raw_inputs, tokenizer)
tokenizer_output = create_text_model_input(tokenizer, token_max_len, raw_inputs)

In [ ]:
from deepsearch.dataset import FlickrDataModule
from deepsearch.metrics import ContrastiveLoss
from deepsearch.model_definition import (ModelComposition,pretrained_text_model,mean_pooling)
from transformers import AutoModel
from torchvision.models import resnet50, ResNet50_Weights

weights = ResNet50_Weights.IMAGENET1K_V1
pretrained_viz_model = resnet50(weights = weights)
preprocess = weights.transforms()

data_mod = FlickrDataModule(df["img_path"], tokenizer_output, 128, 2, True)
criterion = ContrastiveLoss(margin = 0.35, max_violation = False)
model_comp = ModelComposition(pretrained_text_model, pretrained_viz_model, mean_pooling, criterion, 1000, 0.005, 0.5, tokenizer)


In [ ]:
from deepsearch.wandb_logging import setup_wandb

logger, checkpoint_cb = setup_wandb(model_comp, data_mod)

In [ ]:
import pytorch_lightning as pl
import wandb

trainer = pl.Trainer(max_epochs = 1, logger = logger, accelerator = 'auto', devices = 1, log_every_n_steps = 20, callbacks = [checkpoint_cb]) #
trainer.fit(model_comp, data_mod)
wandb.finish()